# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to C:\Users\Josina
[nltk_data]     Baião\AppData\Roaming\nltk_data...


In [2]:
# Read your api key environment variable
load_dotenv(dotenv_path='sentiment.env')

True

In [1]:
# Create a newsapi client

newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

In [ ]:
# Fetch the Bitcoin news articles

bitcoin_headlines = newsapi.get_everything(
    q="bitcoin AND BTC",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Bitcoin: {bitcoin_headlines['totalResults']}")

# Show sample article
bitcoin_headlines["articles"][0]

In [ ]:
# Fetch the Ethereum news articles

ethereum_headlines = newsapi.get_everything(
    q="ethereum AND ETH",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Ethereum: {ethereum_headlines['totalResults']}")

# Show sample article
ethereum_headlines["articles"][0]

In [ ]:
# Create the Bitcoin sentiment scores DataFrame

bitcoin_sentiments = []

for article in bitcoin_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            "text": text      
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
bitcoin_cols = ["Compound", "Negative", "Neutral", "Positive", "text"]
bitcoin_df = bitcoin_df[bitcoin_cols]

bitcoin_df.head()

In [ ]:
# Create the Ethereum sentiment scores DataFrame

ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "Compound": compound,
            "Positive": pos,
            "Negative": neg,
            "Neutral": neu,
            "text": text      
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
ethereum_cols = ["Compound", "Negative", "Neutral", "Positive", "text"]
ethereum_df = ethereum_df[ethereum_cols]

ethereum_df.head()

In [ ]:
# Describe the Bitcoin Sentiment

bitcoin_df.describe()

In [ ]:
# Describe the Ethereum Sentiment

ethereum_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum: 0.77

Q: Which coin had the highest compound score?

A: Bitcoin: 0.89

Q. Which coin had the highest positive score?

A: Bitcoin at 0.30

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [4]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [5]:
# Instantiate the lemmatizer

# Create a list of stopwords

# Expand the default stopwords list if necessary

print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
def tokenizer(text):
    sw = set(stopwords.words('english'))
    
    # Remove the punctuation from text
    
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    
    # Lemmatize Words into root words
    lemmatizer = WordNetLemmatizer()
    lem = [lemmatizer.lemmatize(word) for word in words]
    
    # Convert the words to lowercase
    
    # Remove the punctuation
    token = [word.lower() for word in lem if word.lower() not in sw]
    
    return token

In [ ]:
# Create a new tokens column for Bitcoin

bitcoin_df['tokens'] = bitcoin_df['text'].apply(tokenizer)
bitcoin_df.head()

In [ ]:
# Create a new tokens column for Ethereum

ethereum_df['tokens'] = ethereum_df['text'].apply(tokenizer)
ethereum_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [9]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2

bitcoin_words = tokenizer(''.join(str(bitcoin_df['text'].tolist())))
bitcoin_counts = Counter(ngrams(bitcoin_words, n=2))

print(dict(bitcoin_counts))

In [11]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [12]:
# Function token_count generates the top 10 words for a given coin

def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    
    return Counter(tokens).most_common(N)

In [13]:
# Use token_count to get the top 10 words for Bitcoin

def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Ethereum

token_count(ethereum_words, N=10)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud

bitcoin_string = " ".join(bitcoin_words)
wc = WordCloud().generate(bitcoin_string)
plt.imshow(wc)

In [ ]:
# Generate the Ethereum word cloud

ethereum_string = " ".join(ethereum_words)
ethereum_wc = WordCloud().generate(ethereum_string)
plt.imshow(ethereum_wc)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [19]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model

nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together

bitcoin_concat = str(bitcoin_df['text'].tolist())
bitcoin_concat

In [ ]:
# Run the NER processor on all of the text

bitcoin_doc = nlp(bitcoin_concat)

# Add a title to the document

bitcoin_doc.user_data["title"] = "Bitcoin spaCy Visualization"

In [ ]:
# Render the visualization

displacy.render(bitcoin_doc, style='ent')

In [ ]:
# List all Entities

for ent in bitcoin_doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together

ethereum_concat = str(ethereum_df['text'].tolist())
ethereum_concat

In [ ]:
# Run the NER processor on all of the text

ethereum_doc = nlp(ethereum_concat)

# Add a title to the document

ethereum_doc.user_data["title"] = "Ethereum spaCy Visualization"

In [ ]:
# Render the visualization

displacy.render(ethereum_doc, style='ent')

In [ ]:
# List all Entities

for ent in ethereum_doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

---